# Lightning interface example for a random walk

This notebook contains an example of training the TACTiS model on a random walk dataset.

In [ ]:
import sys
sys.path.append('/Users/nhassen/Documents/ProjectQuant/MyRepos/test/GPT-NeoX-for-Time-Series-Forecasting/')

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
import pytorch_lightning as pl

from GPT_NEO.lightning_module import TradeBotLightning
from GPT_NEO.estimator import TradeBotLightEstimator
from GPT_NEO.estimator import TradeBotLightning

In [ ]:
import torch
from gluon.dataset import generate_backtesting_datasets
from gluon.metrics import compute_validation_metrics
from gluon.plots import plot_four_forecasts
from gluonts.evaluation.backtest import make_evaluation_predictions

In [ ]:
history_factor = 3
backtest_id = 2

metadata, train_data, test_data = generate_backtesting_datasets("electricity_hourly", backtest_id, history_factor, use_cached=False)

In [ ]:
for entry in list(train_data):
    entry["target"] = entry["target"][:20, :]
for entry in list(test_data):
    entry["target"] = entry["target"][:20, :]
entry

In [ ]:
#Original Energy dataset
train_tf_entry = next(iter(list(train_data)))
[k for k in train_tf_entry.keys()]

In [ ]:
list(train_data)[0]

Create Custom Transformation on Energy dataset

In [ ]:
list(train_data)[0]["target"].shape[0]

Create the Lightning version of the TradeBOT model.

The model parameters are almost all in the `model_parameters` dictionary.

In [ ]:

net_estimator = TradeBotLightEstimator(
    num_samples = 100,
    model_parameters= {
        "gamma":0.8,
        "l_norm": 2,
        "data_normalization":"standardization",
        "loss_normalization":"series",
        "series_embedding_dim":13,
        "input_encoder_layers":3,
        "input_encoding_normalization":True,
        "encoder": {
            "attention_layers":3,
            "attention_heads": 3,
            "attention_dim": 4,
            "attention_feedforward_dim": 12,
        },
        "quantile_decoder":{
             "min_u": 0.01,
             "max_u": 0.99,
            "attentional_quantile": {
                "attention_heads": 3,
                "attention_layers": 3,
                "attention_dim": 12,
                "mlp_layers": 3,
                "mlp_dim": 16,
                "resolution": 50,
            },
        }
    },
    learning_rate = 1e-3,
    trainer_kwargs=dict(max_epochs=3, accelerator="cpu"),
    num_series = list(train_data)[0]["target"].shape[0],
    history_length = history_factor * metadata.prediction_length,
    prediction_length = metadata.prediction_length,
    freq = metadata.freq,
    cdf_normalization = True,
    #num_parallel_samples = 100,
)

Train the model. Lightning automatically send the model to GPU if the accelerator is set accordingly, and send it back to CPU after training.

The tuner can automatically find the maximum batch size which can fit in memory, but we give it a maximum number of trials since it does not stop as it goes above the number of samples per epoch.

In [ ]:
predictor = net_estimator.train(train_data)

In [ ]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=test_data, predictor=predictor
)

In [ ]:
trainer.test(datamodule=data_module, ckpt_path='best')

Reduce the batch size parameter, since TACTiS uses a lot of GPU memory during inference due to the many parallel samples.

In [ ]:
data_module.batch_size = 10
predictions = trainer.predict(datamodule=data_module, ckpt_path='best')
predictions = torch.cat(predictions, dim=0)

In [ ]:
timesteps, ground_truths = data_module.predict_groundtruth(include_hist=True)

Plotting the training and validation loss functions during training.

In [ ]:
df = pd.read_csv(os.path.join(logger.log_dir, "metrics.csv"))

plt.figure()
plt.plot(
    df[~df.train_loss.isna()].epoch, df[~df.train_loss.isna()].train_loss, label="train",
)
plt.plot(
    df[~df.valid_loss.isna()].epoch, df[~df.valid_loss.isna()].valid_loss, label="validation",
)
plt.legend()
plt.show()

Plotting a few forecasts.

In [ ]:
def plot_single_series(samples, target, timesteps, index):
    s_samples = samples[index, :, :].cpu().numpy()
    s_timesteps = timesteps[:].cpu().numpy()
    s_target = target[index, :].cpu().numpy()
    
    plt.figure()
    
    for zorder, quant, color, label in [
        [1, 0.05, (0.75,0.75,1), "5%-95%"],
        [2, 0.10, (0.25,0.25,1), "10%-90%"],
        [3, 0.25, (0,0,0.75), "25%-75%"],
    ]:
        plt.fill_between(
            s_timesteps,
            np.quantile(s_samples, quant, axis=1),
            np.quantile(s_samples, 1 - quant, axis=1),
            facecolor=color,
            interpolate=True,
            label=label,
            zorder=zorder,
        )
    
    plt.plot(
        s_timesteps,
        np.quantile(s_samples, 0.5, axis=1),
        color=(0.5,0.5,0.5),
        linewidth=3,
        label="50%",
        zorder=4,
    )
    
    plt.plot(s_timesteps, s_target, color=(0, 0, 0), linewidth=2, zorder=5, label="ground truth")
    
    handles, labels = plt.gca().get_legend_handles_labels()
    order = [0, 1, 2, 3, 4]
    plt.legend([handles[idx] for idx in order], [labels[idx] for idx in order])
    
    plt.show()

In [ ]:
for pred_idx, var_idx in [(13, 3), (5, 7), (2, 8), (16, 0)]:
    plot_single_series(predictions[pred_idx], ground_truths[pred_idx], timesteps[pred_idx], var_idx)